# Домашнее задание №7 - Кластеризация

На этой неделе мы реализуем свою версию алгоритма K-Means, опробуем иерархическую кластеризацию, а также еще раз глянем задачу с клеточными типами на основе данных цитометрии.

In [ ]:
import warnings
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join
from IPython import display
from sklearn.datasets import load_digits
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score # и другие метрики
from sklearn.cluster import KMeans # а также другие алгоритмы

In [ ]:
DATA_PATH = "data"
plt.rcParams["figure.figsize"] = 12, 9
sns.set_style("whitegrid")
warnings.filterwarnings("ignore")

SEED = 111
random.seed(SEED)
np.random.seed(SEED)

## Задание 1. Реализация K-means

**60 баллов**

В данном задании вам предстоит дописать код класса `MyKMeans`. Мы на простом примере увидим, как подбираются центры кластеров и научимся их визуализировать.

(5 баллов) Сгенерируем простой набор данных, 400 объектов и 2 признака (чтобы все быстро работало и можно было легко нарисовать):

In [ ]:
X, true_labels = make_blobs(400, 2, centers=[[0, 0], [-4, 0], [3.5, 3.5], [3.5, -2.0]])

Используем функцию `visualize_clusters`, которая по данным и меткам кластеров будет рисовать их и разукрашивать:

In [ ]:
def visualize_clusters(X, labels):
    """
    Функция для визуализации кластеров
        :param X: таблица объекты х признаки
        :param labels: np.array[n_samples] - номера кластеров
    """
    
    unique_labels = np.sort(np.unique(labels))
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=labels, 
                    palette="colorblind", legend=False,
                    hue_order=unique_labels)
    plt.xlabel("$X_1$", fontsize=18)
    plt.ylabel("$X_2$", fontsize=18)
    
    for label in labels:
        center = X[(labels == label)].mean(axis=0)
        plt.scatter(center[0], center[1], s=80, c="#201F12", marker=(5, 2))

In [ ]:
visualize_clusters(X, true_labels)

(40 баллов) Напишем свой класс `MyKMeans`, который будет реализовывать алгоритм кластеризации K-средних. Напомним сам алгоритм:

1. Выбераем число кластеров (K)
2. Случайно инициализируем K точек (или выбираем из данных), это будут начальные центры наших кластеров
3. Далее для каждого объекта считаем расстояние до всех кластеров и присваиваем ему метку ближайщего
4. Далее для каждого кластера считаем "центр масс" (среднее значение для каждого признака по всем объектам кластера)
5. Этот "центр масс" становится новым центром кластера
6. Повторяем п.3, 4, 5 заданное число итераций или до сходимости

Во время предсказания алгоритм просто находит ближайщий центроид (центр кластера) для тестового объекта и возвращает его номер.

Реализуйте методы:
* `_calculate_distance(X, centroid)` - вычисляет Евклидово расстояние от всех объектов в `Х` до заданного центра кластера (`centroid`)
* `predict(X)` - для каждого элемента из `X` возвращает номер кластера, к которому относится данный элемент

In [ ]:
class MyKMeans:
    def __init__(self, n_clusters, init="random", max_iter=300, visualize=False):
        """
        Конструктор класса MyKMeans
            :param n_clusters: число кластеров
            :param init: способ инициализации центров кластеров
                'random' - генерирует координаты случайно из нормального распределения
                'sample' - выбирает центроиды случайно из объектов выборки
            :param max_iter: заданное число итераций 
                (мы не будем реализовывать другой критерий остановки)
            :param visualize: рисовать ли кластеры и их центроиды в процессе работы
                код будет работать сильно дольше, но красиво...
        """
        
        assert init in ["random", "sample"], f"Неизвестный метод инициализации {init}"
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.centroids = None
        self.visualize = visualize
       
    
    def fit(self, X):
        """
        Подбирает оптимальные центры кластеров
            :param X: наши данные (n_samples, n_features)
        :return self: все как в sklearn
        """
        
        n_samples, n_features = X.shape
        
        # Инициализация центров кластеров
        if self.init == "random":
            centroids = np.random.randn(self.n_clusters, n_features)
        elif self.init == "sample":
            centroids_idx = np.random.choice(np.arange(n_samples), 
                                             size=self.n_clusters, 
                                             replace=False)
            centroids = X[centroids_idx]
        
        # Итеративно двигаем центры
        for _ in range(self.max_iter):
            # Посчитаем расстояния для всех объектов до каждого центроида
            dists = []
            for centroid in centroids:
                dists.append(self._calculate_distance(X, centroid))
            dists = np.concatenate(dists, axis=1)
            # Для каждого объекта найдем, к какому центроиду он ближе
            cluster_labels = np.argmin(dists, axis=1)
            
            # Пересчитаем центр масс для каждого кластера
            centroids = []
            for label in np.sort(np.unique(cluster_labels)):
                center = X[(cluster_labels == label)].mean(axis=0)
                centroids.append(center)
            
            # Отрисуем точки, покрасим по меткам кластера, а также изобразим центроиды
            if self.visualize:
                visualize_clusters(X, cluster_labels)
                display.clear_output(wait=True)
                display.display(plt.gcf())
                plt.close()
                
        self.centroids = np.array(centroids)
        
        return self
    
    
    def predict(self, X):
        """
        Для каждого X возвращает номер кластера, к которому он относится
            :param X: наши данные (n_samples, n_features)
        :return cluster_labels: метки кластеров
        """
        
        """
        YOUR CODE IS HERE
        """
        cluster_labels = None
            
        return cluster_labels
        
        
    def _calculate_distance(self, X, centroid):
        """
        Вычисляет Евклидово расстояние от всех объектов в Х до заданного центра кластера (centroid)
            :param X: наши данные (n_samples, n_features)
            :param centroid: координаты центра кластера
        :return dist: расстояния от всех X до центра кластера
        """
        
        """
        YOUR CODE IS HERE
        """
        dist = None
        
        return dist
    
    
    def __repr__(self):
        return f"Привет, я твой KMeans (/¯◡ ‿ ◡)/¯☆*"

Обучите `MyKMeans` на наших игручешных данных, добейтесь сходимости. Не забудьте поставить `visualize=True`, чтобы посмотреть на красивые картинки. (5 баллов)

Также попробуйте различные способы инициализации центроидов и скажите, какой лучше подошел в этой ситуации? (10 баллов)

## Задание 2. Подбираем лучшую кластеризацию

**60 баллов**

Во 2 и 3 части домашки вам предстоит поработать с данными, котовые вы использовали в ДЗ2. 
Возьмите уже предобработанный вариант из ДЗ2 и загрузите его здесь.

In [ ]:
data = # your preprocessed data from hw2

На лекции были рассмотрены разлинчеы алгоритмы кластеризации. Возьмите три из них (например, `KMeans`, `DBScan` и `AgglomerativeClustering`) и попробуйте понять, какой из алгоритмов будет работать лучше на ваших данных. 

(30 баллов) Для каждого из алгоритмов выберите несколько значений самых важных параметров (не забудьте описать, что за параметры вы выбрали и что они значат в конкретном алгоритме) и посчитайте для них значения `Silhouette` коэффициента. Сначала сделайте это внутри каждого алгоритма, отобрав лучшие гиперпараметры внутри каждого алгоритма. Постройте графики изменения значений силуэта в зависимости от значения параметра и выберите лучший вариант.

(20 баллов) В итоге у вас получится три лучших модели (по одной на каждый алгоритм). Для этих моделей перейдите к сравнению алгоритмов между собой. Сравнение между лучшими проведите при помощи отрисовки `Silhouette plot`, опишите получившиеся результаты и выберите финальный вариант.

(10 баллов) Возьмите любой метод уменьшения размерности (опять же, можете брать тот который на 2дз лучше всего работал или вообще любой) и визуализируйте ваши данные с покраской по лэйблам лучшего алгоритма.

##### YOUR TURN TO CODE

In [ ]:
## ENTER YOUR CODE HERE (/¯◡ ‿ ◡)/¯☆*##

## Задание 3. Рисуем кластермапу

**30 баллов + доп 10 баллов**

Мы также отдельно обсуждали с вами, что есть такая классная вещь как `clustermap`. 

Возьмите ваши исходные данные, отберите из них примерно 50 случайных строк, и отрисуйте `clustermap` для них. Посмотрите на разные метрики расстояния для `clustermap`, попробуйте увидеть какие метрики работают лучше/хуже на вашем датасете (тут просто нарисуйте и посмотрите на дендрограммы по фичам/образцам, сделайте выводы).  

(*доп*) Возьмите метки клеточных типов, которые получались у вас во второй домашке. Используя эти метки сделайте кластермапу с покраской строк по их клеточным типам (тут можно использовать уже все данные). Делается это в seaborn при помощи параметра `row_colors`, не забудьте добавить легенду к вашей визуализации, в которой будет понятно к какому цвету относится тот или иной клеточный тип.

In [2]:
## ENTER YOUR CODE HERE (/¯◡ ‿ ◡)/¯☆*##

**Ваш комментарий к результатам:**

...

На этой неделе кидаю вам мемчик про хитмапы.
С вас тоже что-нибудь забавное:)

Ура, последняя домашка первой части курса сделана! От всей команды поздравляю вас, вы игромные умнички! Было очень классно:)

![pic](https://i.imgflip.com/4wa3y8.jpg)

**Ваши мысли:**

### Therapy time

Напишите здесь ваши впечатления о задании: было ли интересно, было ли слишком легко или наоборот сложно и тд. Также сюда можно написать свои идеи по улучшению заданий, а также предложить данные, на основе которых вы бы хотели построить следующие дз.